In [1]:
import cv2
import numpy as np
from scipy import ndimage
import scipy
from scipy import ndimage
import imageio
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io
import matplotlib.pyplot as plt
import skimage.filters
from numpy import array, zeros, abs, sqrt, arctan2, arctan, pi, real
from numpy.fft import fft2, ifft2


In [2]:
def to_shape(a, shape):
    b = np.zeros(shape)
    b[:a.shape[0], :a.shape[1]] = a
    return b


# non max suppression from https://github.com/fatihsennik/OpenCV-Canny-Edge-Detection/tree/master/code bc no existing function in opencv/scipy
def nonmax_suppression(det, phase):
    gmax = zeros(det.shape)
    for i in range(gmax.shape[0]):
        for j in range(gmax.shape[1]):
            if phase[i][j] < 0:
                phase[i][j] += 360

            if ((j + 1) < gmax.shape[1]) and ((j - 1) >= 0) and ((i + 1) < gmax.shape[0]) and ((i - 1) >= 0):
                # 0 degrees
                if (phase[i][j] >= 337.5 or phase[i][j] < 22.5) or (phase[i][j] >= 157.5 and phase[i][j] < 202.5):
                    if det[i][j] >= det[i][j + 1] and det[i][j] >= det[i][j - 1]:
                        gmax[i][j] = det[i][j]
                # 45 degrees
                if (phase[i][j] >= 22.5 and phase[i][j] < 67.5) or (phase[i][j] >= 202.5 and phase[i][j] < 247.5):
                    if det[i][j] >= det[i - 1][j + 1] and det[i][j] >= det[i + 1][j - 1]:
                        gmax[i][j] = det[i][j]
                # 90 degrees
                if (phase[i][j] >= 67.5 and phase[i][j] < 112.5) or (phase[i][j] >= 247.5 and phase[i][j] < 292.5):
                    if det[i][j] >= det[i - 1][j] and det[i][j] >= det[i + 1][j]:
                        gmax[i][j] = det[i][j]
                # 135 degrees
                if (phase[i][j] >= 112.5 and phase[i][j] < 157.5) or (phase[i][j] >= 292.5 and phase[i][j] < 337.5):
                    if det[i][j] >= det[i - 1][j - 1] and det[i][j] >= det[i + 1][j + 1]:
                        gmax[i][j] = det[i][j]
    return gmax

In [3]:
video = cv2.VideoCapture(0)
 
# Infinite while loop to treat stack of image as video
while True:
    check, frame = video.read()
    
    # im = scipy.signal.medfilt2d(im)
    # im = skimage.filters.gaussian(im, sigma=(sigma, sigma), multichannel=True)


    # #sobel operator
    # # Reading frame(image) from video
    # im = frame.astype('int32')
    # # im = frame.astype('int32')
    # dx = ndimage.sobel(im, 0) # horizontal deriv
    # dy = ndimage.sobel(im, 1) # vertical deriv
    # mag = np.hypot(dx, dy) # magnitude
    # mag *= 255.0/np.max(mag) #normalize

    ###############################################
    # Morphological edge out
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    threshold = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.morphologyEx(threshold, cv2.MORPH_DILATE, kernel)
    morph = cv2.absdiff(dilate, threshold)   
    ###############################################



    ###############################################
    # Canny Edge Detection        
    # cv2.Canny(frame_webcam, 100, 110, edges=outframe)
    canny = cv2.Canny(frame, 100, 110)
    ###############################################


    # noise reduction using median blur
    sigma = 7
    # frame2 = cv2.medianBlur(frame, sigma)
    frame2 = cv2.GaussianBlur(frame, (sigma,sigma), 0)
    im = frame2.astype('int32')
    ###############################################
    # Sobel Edge Detection
    # define filters
    horizontal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])  # s2
    vertical = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])  # s1
    gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

    img_fft = np.fft.fft2(gray_img)
    horizontal_kernel_fft =  np.fft.fft2(to_shape(horizontal, gray_img.shape))
    vertical_kernel_fft =  np.fft.fft2(to_shape(vertical, gray_img.shape) )


    newverticalImage = real(np.fft.ifft2( img_fft * vertical_kernel_fft )).astype(float)
    newhorizontalImage = real(np.fft.ifft2( img_fft * horizontal_kernel_fft )).astype(float)

    

    sobel_fft = abs( np.sqrt(pow(newverticalImage, 2) + pow(newhorizontalImage, 2)) ).astype(float)
    angle = arctan2(newverticalImage, newhorizontalImage) * 180 / pi
    # sobel_fft = nonmax_suppression(sobel_fft, angle)


    # sobel_fft = sobel_fft.astype(np.uint8)

    # sobel_fft = cv2.adaptiveThreshold(sobel_fft,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                 cv2.THRESH_BINARY_INV,7,10)
    
    sobel_fft[sobel_fft<max(sobel_fft.flatten())*.05] = 0

    ###############################################



    ###############################################
    # Prewitt Edge Detection
    # define filters
    horizontal = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])  # s2
    vertical = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])  # s1
    gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

    img_fft = np.fft.fft2(gray_img)
    horizontal_kernel_fft =  np.fft.fft2(to_shape(horizontal, gray_img.shape))
    vertical_kernel_fft =  np.fft.fft2(to_shape(vertical, gray_img.shape) )


    newverticalImage = real(np.fft.ifft2( img_fft * vertical_kernel_fft )).astype(float)
    newhorizontalImage = real(np.fft.ifft2( img_fft * horizontal_kernel_fft )).astype(float)


    prewitt_fft = abs( np.sqrt(pow(newverticalImage, 2) + pow(newhorizontalImage, 2)) ).astype(float)
    angle = arctan2(newverticalImage, newhorizontalImage) * 180 / pi
    # prewitt_fft = nonmax_suppression(prewitt_fft, angle)

    
    prewitt_fft = prewitt_fft.astype(np.uint8)

    # prewitt_fft = cv2.adaptiveThreshold(prewitt_fft,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                 cv2.THRESH_BINARY_INV,7,10)
    
    # prewitt_fft = 255-prewitt_fft
    prewitt_fft[prewitt_fft<max(prewitt_fft.flatten())*.1] = 0
    ###############################################


    ###############################################
    # Laplachian Edge Detection
    # define filters
    horizontal = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]]) #positive laplachian
    # horizontal = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]]) #negative laplachian
    vertical = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]) 
    gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

    img_fft = np.fft.fft2(gray_img)
    horizontal_kernel_fft =  np.fft.fft2(to_shape(horizontal, gray_img.shape))
    vertical_kernel_fft =  np.fft.fft2(to_shape(vertical, gray_img.shape) )


    newverticalImage = real(np.fft.ifft2( img_fft * vertical_kernel_fft )).astype(float)
    newhorizontalImage = real(np.fft.ifft2( img_fft * horizontal_kernel_fft )).astype(float)


    laplachian_fft = abs( np.sqrt(pow(newverticalImage, 2) + pow(newhorizontalImage, 2)) ).astype(float)
    angle = arctan2(newverticalImage, newhorizontalImage) * 180 / pi
    # laplachian_fft = nonmax_suppression(laplachian_fft, angle)

    
    laplachian_fft = laplachian_fft.astype(np.uint8)

    # laplachian_fft = cv2.adaptiveThreshold(laplachian_fft,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                 cv2.THRESH_BINARY_INV,7,10)
    
    # laplachian_fft = 255-prewitt_fft
    laplachian_fft[laplachian_fft<max(laplachian_fft.flatten())*.007] = 0
    ###############################################


    cv2.imshow("normal frame", frame)
    cv2.imshow("edge detection sobel", sobel_fft)
    cv2.imshow("edge detection prewitt", prewitt_fft)
    cv2.imshow("edge detection laplachian", laplachian_fft)
    cv2.imshow("edge detection sobel and laplachian", laplachian_fft)

    cv2.imshow("morphological edge out", morph)
    cv2.imshow("canny edge detection", canny)

   
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()


KeyboardInterrupt: 

In [ ]:
# todo list
# nonmax suppression
# 